This tutorial shows you how to create a pipeline that uses an input LSL stream.

In [ ]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [ ]:
import mindpype as mp
import numpy as np
import pylsl

# This test requires a running LSL stream with the following properties:
# - Type: "EEG"

# This test also requires a marker stream with the following properties:
# - Type: "Markers"
# - Marker Format: "^SPACE pressed$"


First we will specify the channel map and selected channels.

In [ ]:
ch_map =  {'FCz': 0, 'Fz': 1, 'F3': 2, 'F7': 3, 'FC3': 4, 'T7': 5, 'C5': 6, 'C3': 7, 'C1': 8,
        'Cz' : 9, 'CP3': 10, 'CPz': 11, 'P7': 12, 'P5': 13, 'P3': 14, 'P1': 15, 'Pz': 16,
        'PO3': 17, 'Oz': 18, 'PO4': 19, 'P8': 20, 'P6': 21, 'P4': 22, 'P2': 23, 'CP4': 24,
        'T8' : 25, 'C6': 26, 'C4' : 27, 'C2': 28, 'FC4': 29, 'F4': 30, 'F8': 31}

sel_chs = ('FCz',
            'Fz',
            'F3',
            'F7',
            'FC3',
            'T7',
            'C5',
            'C3',
            'C1',
            'Cz',
            'CP3',
            'CPz',
            'P7',
            'P5',
            'P3',
            'P1',
            'Pz',
            'PO3',
            'Oz',
            'PO4',
            'P8',
            'P6',
            'P4',
            'P2',
            'CP4',
            'T8',
            'C6',
            'C4',
            'C2',
            'FC4',
            'F4',
            'F8'
              )

channels = [i for i in range(3,17)]

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline. After creating the session we will create a graph which represents the pipeline.

In [ ]:
# Create a session and graph
session = mp.Session.create()
graph = mp.Graph.create(session)

Next, we will create an LSL input stream using the ```create_marker_coupled_data_stream()``` factory method. We will pass in the session, the stream predicate, the channels, the time offset, and the marker format to create the LSL object.

Then, we will create a tensor from the LSL input stream to serve as one input to our addition node. The LSL stream is volatile so to create the tensor, we will use the ```create_from_handle()``` factory method and pass in our LSL input stream object that we just created. 

We will create a second input tensor using the ```create_from_data()``` factory method.

We will also create a tensor to store our output data. For the output tensor, the library will automatically determine the correct size for the tensor, so we can create a "virtual" tensor, meaning that it can be modified by the library and does not contain any user provided data.

In [ ]:
# Create an LSLInputStream using the factory method
# To create the object, we must pass the session, the stream predicate, the channels, the time offset, and the marker format
# Alternatively, we can pass the data and marker stream infos, which are pylsl.StreamInfo objects

lsl_object = mp.source.InputLSLStream.create_marker_coupled_data_stream(
    session, "type='EEG'",
    channels, 0, marker_fmt="^SPACE pressed$"
    )


# Create a tensor from the LSLInputStream. Since the LSL stream is volatile (not static) between trials,
# we create the tensor from "handle", passing the stream object and the shape of the tensor
t_in = mp.Tensor.create_from_handle(session, (len(channels), 1), lsl_object)

# Create a second tensor from data, passing the session, the shape, and the data
t_in_2 = mp.Tensor.create_from_data(session, data=np.zeros(t_in.shape))

# Create an output tensor
t_out = mp.Tensor.create_virtual(session)



Next, we will add our addition node to the graph using the ```add_to_graph()``` factory method.

In [ ]:
# Add an addition node to the graph, passing the graph, the input tensors, and the output tensor
Add = mp.kernels.AdditionKernel.add_to_graph(graph, t_in, t_in_2, t_out)

At this point we have created the processing pipeline, and added all of the required components. We can now use a graph method to verify that the structure of the pipeline is valid.

Following the verification of the pipeline, we should now initialize the graph. This step is required for pipelines that have methods that need to be trained or fit (ie. classifiers), but optional for other pipelines.


In [ ]:
# Verify and initialize the graph
graph.verify()
graph.initialize()

Finally, we will execute the graph 10 times, printing each sum output.

In [ ]:
for i in range(10):
    # Execute the graph. The execute method will automatically wait for the correct marker
    sts = graph.execute()
    print(t_out.data)
